### Calculate Slope and Aspect

With this example, you can calculate slope and aspect data from elevation data. Before start, we create mosaic file for calculation part. If your AOI is too big and you don't have enough disc space, You should put all steps in one loop and end of the calculation you can delete mosaic file.

In [17]:
from osgeo import gdal,ogr
import pandas as pd
import geopandas
import os

In [18]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return sorted(FilesList),sorted(FileName)
        else:
            print("no"+ extension +"file for this directory")

In [19]:
def ImageBoundry(FilePath):
        #for aspect data
        imgname=FilePath

        from osgeo import gdal,ogr
        ds = gdal.Open(FilePath)
        gt = ds.GetGeoTransform()  # captures origin and pixel size

        ULC = gdal.ApplyGeoTransform(gt, 0, 0) #Upper Left Corner
        URC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, 0) #Upper Right Corner
        LLC = gdal.ApplyGeoTransform(gt, 0, ds.RasterYSize) #Lower Left Corner
        LRC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, ds.RasterYSize) #Lower Right Corner
                   
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint_2D(ULC[0], ULC[1])
        ring.AddPoint_2D(URC[0], URC[1])
        ring.AddPoint_2D(LRC[0], LRC[1])
        ring.AddPoint_2D(LLC[0], LLC[1])
        ring.AddPoint_2D(ULC[0], ULC[1]) 
        poly=ogr.Geometry(ogr.wkbPolygon)
        poly.AddGeometry(ring)
        poly.ExportToJson
        return poly.ExportToJson() ,imgname
    


In [20]:
# Target  elevation data. We get path and name of the files
ImageList=list(ListofExtensionAndName(r"./data//elevation_tif/",".tif"))

In [23]:
# With ImageBoundry function, we create geopandas dataFrame and put the extend of tif image
ImgboundaryList=[]
gdf=geopandas.geodataframe.GeoDataFrame()
for i in ImageList[0]:
    imgboundary=ImageBoundry(i)
    gdf_new = geopandas.read_file(imgboundary[0])
    ImgboundaryList.append(i)
    gdf=gdf.append(gdf_new)


In [24]:
# main folder of your download
target_folder='./data/'
elevation_mosaic=target_folder+'elevation_mosaic/'
if not os.path.isdir(elevation_mosaic):
    os.mkdir(elevation_mosaic)

In [25]:
Mosaiclist=[]
imglist=[]


In [27]:
for i in ImageList[0]:
    #TARGET IMAGE INDEX
    imgindex=ImageList[0].index(i)
    # Define output mosaic name
    mosaic_name=ImageList[1][imgindex]
    
    # get target image boundry
    imgboundary2=ImageBoundry(i)
    gdf_mosaic = geopandas.read_file(imgboundary2[0])
    # we get geometry feature for calculating distance
    gdfmosaic_geo=gdf_mosaic.geometry
    # we define the neighbours
    dist=gdfmosaic_geo.distance(gdf.geometry)
    for index,j in enumerate(dist,0):
        # if distance smaller than 0.5, image is neigbour of the target image
        if j <0.5:
            imglist.append(ImgboundaryList[index])
            
    # we save the mosaic lists. It could be useful       
    Mosaiclist.append(imglist)
    mosaicpath=elevation_mosaic+mosaic_name+'.tif'
    gdal.Warp(mosaicpath,imglist)
    imglist=[]
     

In [28]:
Input=r"../geoserver_data/2020_Elevation_Slope_Aspect/elevation_mosaic"

OutputUTM=r"../geoserver_data/2020_Elevation_Slope_Aspect/elevation_utm_mosaic"

Input_MozaikList=ListofExtensionAndName(Input,"tif")


In [31]:
# main folder of your download
target_folder='./data/'
elevation_mosaic=target_folder+'elevation_mosaic/'
# Elevation files (UTM) location
elevation_utm_mosaic=target_folder+'elevation_utm_mosaic/'
if not os.path.isdir(elevation_utm_mosaic):
    os.mkdir(elevation_utm_mosaic)
Input_MozaikList=ListofExtensionAndName(elevation_mosaic,"tif")
sh_file='./mosaic_to_utm.sh'

### Convert .tif file to EPSG:3857

In [32]:
for index,i in enumerate(Input_MozaikList[0],0):

    ImgOutPath=elevation_utm_mosaic+"/"+Input_MozaikList[1][index]+'.tif'
    dest=i
    #print(ImgOutPath)
    gdal.Warp(ImgOutPath, dest, format = 'GTiff', dstSRS = 'EPSG:3857')
    
    

#### Below and above codes are doing same job.


In [16]:
# Preparing linux cmd code to converting tif files
for index,i in enumerate(Input_MozaikList[0],0):
    outputimg=elevation_utm_mosaic+"/"+Input_MozaikList[1][index]+".tif"
    #print(outputimg)
    translate_code= "gdalwarp -s_srs EPSG:4326 -t_srs EPSG:3857 -r near -of GTiff"+' '+i+' '+outputimg
    #print(slope_code)
    with open(sh_file, 'a') as the_file:
        the_file.write(translate_code+"\n")    

## Slope calculation


In [35]:
# Main folder of your download
target_folder='./data/'
# Elevation files (UTM) location
elevation_utm_mosaic=target_folder+'elevation_utm_mosaic/'
slope_utm_mosaic=target_folder+'slope_utm_mosaic/'
if not os.path.isdir(slope_utm_mosaic):
    os.mkdir(slope_utm_mosaic)
Slope_Input_MozaikList=ListofExtensionAndName(elevation_utm_mosaic,"tif")
# be careful when you declare bash file location. It should be same place as Jupyter file.
slope_sh_file='./slope_calculate.sh'

In [43]:
'''
For all algorithms, except color-relief, a nodata value in the target dataset will be emitted if at least one pixel set to
the nodata value is found in the 3x3 window centered around each source pixel. The consequence is that there will
be a 1-pixel border around each image set with nodata value. From GDAL 1.8.0, if -compute_edges is specified,
gdaldem will compute values at image edges or if a nodata value is found in the 3x3 window, by interpolating
missing values.

https://gis-lab.info/docs/gdal/gdal_ogr_user_docs.pdf
'''

'\nFrom GDAL 1.8.0, if -compute_edges is specified, gdaldem will compute values at image edges or\nif a nodata value is found in the 3x3 window, by interpolating missing values.\n'

In [36]:

# After this task, You should go to directory in command line and start job
# with >  bash slope_calculate.sh 
for index,i in enumerate(Slope_Input_MozaikList[0],0):
    outputimg=slope_utm_mosaic+"/"+Slope_Input_MozaikList[1][index]+'_Slope'".tif"
    #print(outputimg)
    slope_code= "gdaldem slope"+ " "+ i + " "+ outputimg + " "+ "-of GTiff -b 1 -s 1.0 -compute_edges"
    with open(slope_sh_file, 'a') as the_file:
        the_file.write(slope_code+"\n")

    

### Export the Slope Result

In [40]:
def ImageBoundry_for_Gdal(FilePath):     
        from osgeo import gdal,ogr
        ds = gdal.Open(FilePath)
        gt = ds.GetGeoTransform()  # captures origin and pixel size
        ULC = gdal.ApplyGeoTransform(gt, 0, 0) #Upper Left Corner
        URC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, 0) #Upper Right Corner
        LLC = gdal.ApplyGeoTransform(gt, 0, ds.RasterYSize) #Lower Left Corner
        LRC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, ds.RasterYSize) #Lower Right Corner                   
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint_2D(ULC[0], ULC[1])
        ring.AddPoint_2D(URC[0], URC[1])
        ring.AddPoint_2D(LRC[0], LRC[1])
        ring.AddPoint_2D(LLC[0], LLC[1])
        ring.AddPoint_2D(ULC[0], ULC[1]) 
        poly=ogr.Geometry(ogr.wkbPolygon)
        poly.AddGeometry(ring)
        poly.ExportToJson
        return ULC[0],ULC[1],LRC[0],LRC[1]

In [38]:
Slope_UTM_MosaicList=ListofExtensionAndName(slope_utm_mosaic,'.tif')
#output of slope result
SlopeSubset=target_folder+'slope_result_4326/'
if not os.path.isdir(SlopeSubset):
    os.mkdir(SlopeSubset)
# we use this folder temporarily because end of the process we delete mosaic_4326 image 
#because of hardisk issue
slope_mosaic_4326=target_folder+'slope_mosaic_4326/'
if not os.path.isdir(slope_mosaic_4326):
    os.mkdir(slope_mosaic_4326)
# Get extend of raw images
RawImagePath=target_folder+'elevation_tif/'
rawimg_list=ListofExtensionAndName(RawImagePath,'.tif')

In [41]:
boundry_rawimg=[]
for i in rawimg_list[0]:
    boundry_raw=ImageBoundry_for_Gdal(i)
    boundry_rawimg.append(boundry_raw)


##### Cropping slope data

For this task, we create only one 'for' loop. Convert slope result(UTM) to epsg:4326 and crop data according to raw data boundry

In [42]:
#create 4326 mosaic,subset and delete
for index,i in enumerate(Slope_UTM_MosaicList[0],0):

    ImgOutPath=slope_mosaic_4326+Slope_UTM_MosaicList[1][index]+'.tif'
    gdal.Warp(ImgOutPath, i, format = 'GTiff', dstSRS = 'EPSG:4326')
    
    Subset_ImgOutPath=SlopeSubset+Slope_UTM_MosaicList[1][index]+'.tif'
    print(Subset_ImgOutPath)
    boundry=boundry_rawimg[index]
    
    ds1 = gdal.Open(ImgOutPath)
    dstDS = gdal.Warp(Subset_ImgOutPath,[ds1], format = 'GTiff',outputBounds=[boundry[0],boundry[3],boundry[2],boundry[1]],
                      dstSRS = 'EPSG:4326',resampleAlg = gdal.GRIORA_NearestNeighbour,
                      outputType=gdal.GDT_Float32,width = 6001,height = 6001)
    
    os.remove(ImgOutPath)

./data/slope_result_4326/EarthEnv-DEM90_N40W005_Slope.tif
./data/slope_result_4326/EarthEnv-DEM90_N40W010_Slope.tif


## Aspect Calculation

We use same flowchart to calculate the aspect. Be careful defining path

In [ ]:
# Main folder of your download
target_folder='./data/'
# Elevation files (UTM) location
elevation_utm_mosaic=target_folder+'elevation_utm_mosaic/'
aspect_utm_mosaic=target_folder+'aspect_utm_mosaic/'
if not os.path.isdir(aspect_utm_mosaic):
    os.mkdir(aspect_utm_mosaic)
Aspect_Input_MozaikList=ListofExtensionAndName(elevation_utm_mosaic,"tif")
aspect_sh_file=target_folder+'aspect_calculate.sh'

In [ ]:
# After this task, You should go to directory in command line and start job
# with >  bash aspect_calculate.sh 
for index,i in enumerate(Aspect_Input_MozaikList[0],0):
    outputimg=aspect_utm_mosaic+Aspect_Input_MozaikList[1][index]+'_Aspect'".tif"
    print(outputimg)
    aspect_code= "gdaldem aspect"+ " "+ i + " "+ outputimg + " "+ "-of GTiff -b 1 -zero_for_flat -compute_edges"
    with open(aspect_sh_file, 'a') as the_file:
        the_file.write(aspect_code+"\n")

    

In [ ]:
Aspect_UTM_MosaicList=ListofExtensionAndName(aspect_utm_mosaic)
#output of aspect result
AspectSubset=target_folder+'aspect_result_4326/'
if not os.path.isdir(AspectSubset):
    os.mkdir(AspectSubset)
# we use this folder temporarily because end of the process we delete mosaic_4326 image 
#because of hardisk issue
aspect_mosaic_4326=target_folder+'aspect_mosaic_4326/'
if not os.path.isdir(aspect_mosaic_4326):
    os.mkdir(aspect_mosaic_4326)
# Get extend of raw images
RawImagePath=target_folder+'elevation_tif/'
rawimg_list=ListofExtensionAndName(RawImagePath,'.tif')

#### At the same time,If you calculate slope, no need to run this code

In [ ]:
boundry_rawimg=[]
for i in rawimg_list[0]:
    boundry_raw=ImageBoundry_for_Gdal(i)
    boundry_rawimg.append(boundry_raw)

In [ ]:
#create 4326 mosaic,subset and delete
for index,i in enumerate(Aspect_UTM_MosaicList[0],0):

    ImgOutPath=slope_mosaic_4326+Aspect_UTM_MosaicList[1][index]+'.tif'
    gdal.Warp(ImgOutPath, i, format = 'GTiff', dstSRS = 'EPSG:4326')
    
    Subset_ImgOutPath=SlopeSubset+Aspect_UTM_MosaicList[1][index]+'.tif'
    print(Subset_ImgOutPath)
    boundry=boundry_rawimg[index]
    
    ds1 = gdal.Open(ImgOutPath)
    dstDS = gdal.Warp(Subset_ImgOutPath,[ds1], format = 'GTiff',outputBounds=[boundry[0],boundry[3],boundry[2],boundry[1]],
                      dstSRS = 'EPSG:4326',resampleAlg = gdal.GRIORA_NearestNeighbour,
                      outputType=gdal.GDT_Float32,width = 6001,height = 6001)
    
    os.remove(ImgOutPath)